In [3]:
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
import sounddevice as sd
import pandas as pd
import os
import sys
import copy
sys.path.append(r"C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound")
from ecosound.core.measurement import Measurement
from ecosound.core.annotation import Annotation
from ecosound.core.audiotools import Sound
from ecosound.core.spectrogram import Spectrogram
from ecosound.visualization.grapher_builder import GrapherFactory
pn.extension()

# load dataset
data_file=r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\results\dataset_FS-NN.nc'
dataset = Measurement()
dataset.from_netcdf(data_file)
data_all = dataset.data
data = copy.deepcopy(data_all)
#data = data.sample(n = 10000)

# Widgets
inputs = list(data.columns)
X_variable  = pn.widgets.Select(name='X axis',
                                value='frequency_min',
                                options=inputs)
Y_variable  = pn.widgets.Select(name='Y axis',
                                value='frequency_max',
                                options=inputs)
Color  = pn.widgets.Select(name='Color',
                           value='label_class',
                           options=inputs)
alpha_slider = pn.widgets.FloatSlider(name='Point transparency',
                                      start=0, end=1,
                                      step=0.1,
                                      value=0.2,
                                     )
size_slider = pn.widgets.IntSlider(name='Point size',
                                   start=1,
                                   end=40,
                                   step=1,
                                   value=6,
                                  )

sampling_slider = pn.widgets.FloatSlider(name='% data loaded',
                                      start=0, end=100,
                                      step=10,
                                      value=20,
                                        )

markdown = pn.pane.Markdown("<b>Markdown display</b>", width=400)

sound_checkbox = pn.widgets.Checkbox(name='Automatically play selected sound', value=True)

# Color value multi selector
color_values_multi_select = pn.widgets.MultiSelect(name='Selected color values', value= list(data[Color.value].unique()), options=list(data[Color.value].unique()), size=8)
def callback_color_selection(target, event):
    target.options = list(data_all[event.new].unique())#    target.value = list(data_all[event.new].unique())  # selects everything by default.
Color.link(color_values_multi_select, callbacks={'value': callback_color_selection})

# Stream
tap = hv.streams.Selection1D(index=[data.index.min()])

# multi selector for user Selection 
selection_multi_select = pn.widgets.MultiSelect(name='Selected points', value= [data.index.min()], options=[data.index.min()], size=8)
def callback_points_selection(*events):
    for event in events:
        if event.name == 'index':
            if event.new:
                selection_multi_select.options = event.new
                if selection_multi_select.options:
                    selection_multi_select.value = [selection_multi_select.options[0]]
watcher = tap.param.watch(callback_points_selection, ['index'], onlychanged=False)

# Subsample
def subsample(data1, sampling_slider):
    n_points = int(len(data1)*sampling_slider)
    return data1.sample(n = n_points)

# Filter color values based on multi selector
def filter_color_values(data1,Color, color_values_multi_select):
    return data1[data1[Color].isin(color_values_multi_select)]

# Scatter plot
@pn.depends(X_variable, Y_variable, Color, alpha_slider, size_slider,sampling_slider, color_values_multi_select)
def scatterplot(X_variable, Y_variable, Color, alpha_slider, size_slider,sampling_slider, color_values_multi_select):
    global data
    # subsampling (from slider)
    data = subsample(data_all, sampling_slider/100)
    # filter based on color values selected
    if len(color_values_multi_select)>0:
        data = filter_color_values(data, Color, color_values_multi_select)
    # Scatter plot
    scatter_plot = data.hvplot.scatter(x=X_variable,
                                       y=Y_variable,
                                       c=Color,
                                       s=size_slider,
                                       alpha=alpha_slider,
                                       title= str(len(data)) + '/' + str(len(data_all)) + ' points',
                                       #datashade=True,
                                       tools=['tap', 'box_select', 'lasso_select'],
                                       #active_tools=['wheel_zoom'],
                                       hover_cols = ['index','uuid'],
                                       responsive=True,
                                       nonselection_alpha = 0.1,
                                       selection_alpha = 1,
                                       min_width=800,
                                       min_height=450,
                                      )
    tap.source = scatter_plot   
    #scatter_plot.opts(click_policy="hide")
    return scatter_plot 

# Measurement table
@pn.depends(tap.param.index)
def table(index):  
    if index:
        return pn.pane.DataFrame(data.iloc[index[0]])

    
# Spectrogram
@pn.depends(index=selection_multi_select, play_sound=sound_checkbox)
def spectrogram_plot(index, play_sound):
    if index:
        frame = 0.0625 #3000
        nfft = 0.0853 # 4096
        step = 0.01 # 5
        fmin = 0
        fmax = 1000
        window_type = 'hann'
        time_buffer = 1
        palet = 'jet' # 'binary'

        data_selection = data.iloc[index[0]]
        index_selection = data.index[index[0]]

        wavfilename = os.path.join(data_selection.audio_file_dir, data_selection.audio_file_name + data_selection.audio_file_extension)
        t1 = data_selection.time_min_offset - time_buffer
        t2 = data_selection.time_max_offset + time_buffer
        # load audio data
        sound = Sound(wavfilename)
        sound.read(channel=0, chunk=[t1, t2], unit='sec', detrend=True)
        # Calculates  spectrogram
        spectro = Spectrogram(frame, window_type, nfft, step, sound.waveform_sampling_frequency, unit='sec')
        spectro.compute(sound, dB=True, use_dask=False, dask_chunks=40)
        # Define annotation box
        annot = Annotation()
        annot.data = annot.data.append({'time_min_offset': time_buffer,
                                        'time_max_offset': time_buffer + data_selection.duration,
                                        'frequency_min': data_selection.frequency_min,
                                        'frequency_max': data_selection.frequency_max,
                                        'duration':data_selection.duration,
                                       },
                                       ignore_index=True)

        # Plot
        graph = GrapherFactory('SoundPlotter', title=str(index_selection) + ': ' +data_selection.label_class + ' - ' +data_selection.label_subclass, frequency_max=fmax)
        graph.add_data(spectro)
        graph.add_annotation(annot, panel=0, color='black', label='Detections')

        #graph.colormap = 'binary'
        graph.colormap = palet
        fig, ax = graph.show(display=False)

        # play sound
        if play_sound:
            sd.play(sound.waveform/max(sound.waveform), sound.waveform_sampling_frequency)
        
        return pn.pane.Matplotlib(fig)


# Dashboard
#gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=800)
tabs = pn.Tabs(('Axes',pn.Column(X_variable, Y_variable, Color, color_values_multi_select)))
tabs.append(('Style',pn.Column(alpha_slider, size_slider)))
widgets = pn.Column(sampling_slider, tabs)
tabs2 = pn.Tabs(('Spectrogram',spectrogram_plot))
tabs2.append(('Measurements',table))

dashboard = pn.Column(pn.Row(widgets, scatterplot),pn.Row(tabs2, pn.Column(selection_multi_select,sound_checkbox)))
dashboard.show()
#dashboard


label_class
['NN', 'FS']
Launching server at http://localhost:49723


label_class
['NN', 'FS']
label_class
['NN', 'FS']
label_class
['NN', 'FS']
label_class
['NN', 'FS']
(Event(what='value', name='index', obj=Selection1D(index=[8318]), cls=Selection1D(index=[8318]), old=[0], new=[8318], type='set'),)
(Event(what='value', name='index', obj=Selection1D(index=[4500, 6625, 4751, 4983, 315]), cls=Selection1D(index=[4500, 6625, 4751, 4983, 315]), old=[8318], new=[4500, 6625, 4751, 4983, 315], type='set'),)
(Event(what='value', name='index', obj=Selection1D(index=[11383, 11657, 9777]), cls=Selection1D(index=[11383, 11657, 9777]), old=[4500, 6625, 4751, 4983, 315], new=[11383, 11657, 9777], type='set'),)
(Event(what='value', name='index', obj=Selection1D(index=[]), cls=Selection1D(index=[]), old=[11383, 11657, 9777], new=[], type='set'),)
(Event(what='value', name='index', obj=Selection1D(index=[10908]), cls=Selection1D(index=[10908]), old=[], new=[10908], type='set'),)
(Event(what='value', name='index', obj=Selection1D(index=[217, 5012, 11508, 3734, 10167, 3715

C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


(Event(what='value', name='index', obj=Selection1D(index=[]), cls=Selection1D(index=[]), old=[3635, 897, 6458, 4100, 6756, 3253, 3890, 5366, 4188, 7548, 8200, 6630, 7625], new=[], type='set'),)
(Event(what='value', name='index', obj=Selection1D(index=[234, 1376, 1984, 3203, 3548, 3897, 3917, 5524, 6111, 6710, 7883]), cls=Selection1D(index=[234, 1376, 1984, 3203, 3548, 3897, 3917, 5524, 6111, 6710, 7883]), old=[], new=[234, 1376, 1984, 3203, 3548, 3897, 3917, 5524, 6111, 6710, 7883], type='set'),)
(Event(what='value', name='index', obj=Selection1D(index=[234, 1376, 1404, 1793, 1984, 2320, 3203, 3280, 3548, 3799, 3897, 3917, 4333, 5524, 6111, 6710, 7211, 7788, 7883, 7942]), cls=Selection1D(index=[234, 1376, 1404, 1793, 1984, 2320, 3203, 3280, 3548, 3799, 3897, 3917, 4333, 5524, 6111, 6710, 7211, 7788, 7883, 7942]), old=[234, 1376, 1984, 3203, 3548, 3897, 3917, 5524, 6111, 6710, 7883], new=[234, 1376, 1404, 1793, 1984, 2320, 3203, 3280, 3548, 3799, 3897, 3917, 4333, 5524, 6111, 6710, 7211

C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


...

C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


(Event(what='value', name='index', obj=Selection1D(index=[4111, 150, 652, 167, 8281, 4898, 3240, 4097, 5350, 111, 7654, 6294, 4743, 1505, 695, 1201, 7502, 8072, 4720, 674, 8690, 6063, 5554, 6351, 2931, 5025, 6189]), cls=Selection1D(index=[4111, 150, 652, 167, 8281, 4898, 3240, 4097, 5350, 111, 7654, 6294, 4743, 1505, 695, 1201, 7502, 8072, 4720, 674, 8690, 6063, 5554, 6351, 2931, 5025, 6189]), old=[979, 6357, 3474, 7812, 1870, 2399], new=[4111, 150, 652, 167, 8281, 4898, 3240, 4097, 5350, 111, 7654, 6294, 4743, 1505, 695, 1201, 7502, 8072, 4720, 674, 8690, 6063, 5554, 6351, 2931, 5025, 6189], type='set'),)
(Event(what='value', name='index', obj=Selection1D(index=[6420, 1738, 4104]), cls=Selection1D(index=[6420, 1738, 4104]), old=[4111, 150, 652, 167, 8281, 4898, 3240, 4097, 5350, 111, 7654, 6294, 4743, 1505, 695, 1201, 7502, 8072, 4720, 674, 8690, 6063, 5554, 6351, 2931, 5025, 6189], new=[6420, 1738, 4104], type='set'),)


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,
